<a href="https://colab.research.google.com/github/NataliaGon/kpi/blob/math-models/Lab3.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
import os
import numpy as np
import pandas as pd
from sklearn.model_selection import GridSearchCV
from sklearn.preprocessing import LabelEncoder, StandardScaler
from sklearn import datasets
from sklearn.model_selection import train_test_split
from sklearn.svm import SVC
from sklearn.metrics import accuracy_score
import kagglehub

In [2]:
path = kagglehub.dataset_download("whenamancodes/infoseccyber-security-salaries")

print("Path to dataset files:", path)
print("Files in dataset folder:", os.listdir(path))

Path to dataset files: /root/.cache/kagglehub/datasets/whenamancodes/infoseccyber-security-salaries/versions/1
Files in dataset folder: ['Cyber_salaries.csv']


In [3]:
df = pd.read_csv(os.path.join(path, "Cyber_salaries.csv"))

print("Size rows/columns:", df.shape)

print(df.isnull().sum())

df.describe()


Size rows/columns: (1349, 11)
work_year             0
experience_level      0
employment_type       0
job_title             0
salary                0
salary_currency       0
salary_in_usd         0
employee_residence    0
remote_ratio          0
company_location      0
company_size          0
dtype: int64


,work_year,salary,salary_in_usd,remote_ratio
count,1349.000000,1.349000e+03,1349.000000,1349.000000
mean,2021.368421,5.288248e+05,121578.622683,72.238695
std,0.711236,1.361367e+07,68972.954838,39.355331
min,2020.000000,1.740000e+03,2000.000000,0.000000
25%,2021.000000,8.000000e+04,75000.000000,50.000000
50%,2022.000000,1.200000e+05,115000.000000,100.000000
75%,2022.000000,1.600800e+05,152000.000000,100.000000
max,2022.000000,5.000000e+08,899920.000000,100.000000


In [4]:
target = "experience_level"
drop_cols = [
    "salary","salary_currency"
]


X = df.drop(columns=drop_cols + [target])
y = df[target]

for col in X.select_dtypes(include=["object"]).columns:
    X[col] = LabelEncoder().fit_transform(X[col])


scaler = StandardScaler()
X[["salary_in_usd", "remote_ratio"]] = scaler.fit_transform(X[["salary_in_usd", "remote_ratio"]])

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2,
random_state=42)

print("Train size:", X_train.shape)
print("Test size:", X.describe())

Train size: (1079, 8)
Test size:          work_year  employment_type    job_title  salary_in_usd  \
count  1349.000000      1349.000000  1349.000000    1349.000000   
mean   2021.368421         1.988139    43.484062       0.000000   
std       0.711236         0.203475    23.353002       1.000371   
min    2020.000000         0.000000     0.000000      -1.734346   
25%    2021.000000         2.000000    19.000000      -0.675568   
50%    2022.000000         2.000000    46.000000      -0.095415   
75%    2022.000000         2.000000    68.000000       0.441226   
max    2022.000000         3.000000    86.000000      11.288918   

       employee_residence  remote_ratio  company_location  company_size  
count         1349.000000  1.349000e+03       1349.000000   1349.000000  
mean            46.836916  1.158780e-16         45.170497      0.476649  
std             17.231883  1.000371e+00         15.504206      0.621400  
min              0.000000 -1.836231e+00          0.000000      0.00

In [5]:
clf = SVC(kernel='linear') #Параметр kernel='linear' створює пряму або площину для розділення класів і підходить для лінійно роздільних даних.
clf.fit(X_train, y_train)
y_pred = clf.predict(X_test)

accuracy = accuracy_score(y_test, y_pred)

print('Accuracy', accuracy)

print("Train class distribution:\n", y_train.value_counts(normalize=True))
print(f"\n================================")
print("Test  class distribution:\n", y_test.value_counts(normalize=True))


Accuracy 0.5481481481481482
Train class distribution:
 experience_level
SE    0.467099
MI    0.316033
EN    0.164041
EX    0.052827
Name: proportion, dtype: float64

Test  class distribution:
 experience_level
SE    0.437037
MI    0.340741
EN    0.151852
EX    0.070370
Name: proportion, dtype: float64


Отримано початкову модель з точністю близько 54–55%. Розподіл класів показав значну дисбалансованість (найменше представлений клас EX ≈ 5%).
Це пояснює середній рівень точності, адже модель гірше класифікує рідкісні категорії.

In [6]:
param_grid = {
     'kernel': ['rbf'], #kernel='rbf' формує криву межу, що дозволяє моделі класифікувати більш складні, нелінійні залежності.
     'C': [0.1, 1, 10], # Параметр C керує жорсткістю розділення: менше значення (0.1) дає ширшу межу та більше помилок, більше (10) — вже межу та менше помилок, але ризик переобучення.
     'gamma': ['scale', 0.01, 0.1], # Параметр gamma визначає, наскільки сильно один об’єкт впливає на інші: малі значення (0.01) створюють плавну межу, великі (0.1) — більш вигнуту та локальну.
}

grid = GridSearchCV(SVC(), param_grid, cv=5, scoring='accuracy')
grid.fit(X_train, y_train)

print("Best parameters:", grid.best_params_)
print("Best CV accuracy:", grid.best_score_)

Best parameters: {'C': 10, 'gamma': 0.1, 'kernel': 'rbf'}
Best CV accuracy: 0.5968475452196382


Після оптимізації гіперпараметрів SVM із використанням ядра RBF модель досягла найкращих результатів при параметрах:
C = 10, gamma = 0.1, kernel = 'rbf'.

Отримана середня точність крос-перевірки (CV accuracy) становить ≈ 0.597, що є вищою, ніж попередній результат із лінійним ядром (0.55).
Це підтверджує, що нелінійне RBF-ядро краще відображає складні зв’язки між ознаками та рівнем досвіду працівників.